In [152]:
import pandas as pd
import numpy as np
#Importación de datos
df_precios = pd.read_csv('files_folder/precios_trimestre.csv')
df_propiedades = pd.read_csv('files_folder/catalogo_inmobiliario.csv')
df_ciudades = pd.read_csv('files_folder/info_ciudades.csv')

print("Info de precios trimestre:")
print(df_precios)
print("Info de propiedades trimestre:")
print(df_propiedades)
print("Info de ciudades trimestre:")
print(df_ciudades)

# No se hace conversión a DataFrame, ya que cuando se hace la lectura, estos automáticamente pasa a ser un dataframe

# Se crea una copia
data_price=df_precios.copy()
data_properties=df_propiedades.copy()
data_cities=df_ciudades.copy()


Info de precios trimestre:
  id_propiedad  Ene_2024  Feb_2024  Mar_2024
0      INM-001     350.0       355       360
1      INM-002     420.0       415       410
2      INM-003       NaN       210       215
3      INM-004     800.0       790       810
4      INM-005     120.0       120       125
Info de propiedades trimestre:
         ID       descripcion_texto  area_metros
0   inm-001    Apartamento | Bogota           65
1   inm-002         Casa | Medellin          120
2   INM-003      Apartamento | Cali           50
3  inm-004            Casa | Bogota          250
4   INM-005  Apartamento | Medellin           45
Info de ciudades trimestre:
  ciudad_ref  estrato_promedio riesgo_inversion
0     Bogota                 4             Bajo
1   Medellin                 5            Medio
2       Cali                 3             Alto
3  Cartagena                 6             Bajo


In [153]:
# Reorganiza el DataFrame de formato ancho a largo:
# mantiene el identificador de propiedad y convierte las columnas de ventas mensuales
# en filas con dos nuevas columnas: 'Mes' y 'Ventas'
data_price = pd.melt(
    data_price,
    id_vars=['id_propiedad'],          # Columnas que se mantienen
    value_vars=['Ene_2024', 'Feb_2024','Mar_2024'], # Columnas a convertir en filas
    var_name='Mes',          # Nombre para la nueva columna de variables
    value_name='Precios'      # Nombre para la nueva columna de valores
)

In [154]:
# Se verifico que hay un valor 'NaN', se elimina porque puede interferir con el análisis posterior, ya que no se tiene información de ventas para esa propiedad en ese mes
data_price['Precios'] = data_price['Precios'].fillna(0)

In [155]:
# 1. Limpieza de la columna 'ID'
# Convertimos todos los valores a mayúsculas y eliminamos espacios innecesarios
data_properties['ID'] = data_properties['ID'].str.upper().str.strip()

# 2. Limpieza de espacios en todas las columnas de tipo objeto (strings)
data_properties = data_properties.apply(
    lambda x: x.str.strip() if x.dtype == "object" else x
)

# 3. División de la columna 'descripción_texto' en nuevas columnas
info_separada = data_properties['descripcion_texto'].str.split('|', expand=True)

# 4. Asignación de los nuevos nombres de columnas
data_properties['Tipo_Propiedad'] = info_separada[0].str.strip().str.title()
data_properties['Ciudad'] = info_separada[1].str.strip()


In [156]:
# 5. Eliminación de la columna original 'descripción_texto', se hace en otra celda para que no corra errores cuando se corra esta celda de nuevo de manera independiente
data_properties = data_properties.drop(columns=['descripcion_texto'])

# 6. Mostrar los datos limpios
print("Datos de propiedades limpias:")
print(data_properties)

Datos de propiedades limpias:
        ID  area_metros Tipo_Propiedad    Ciudad
0  INM-001           65    Apartamento    Bogota
1  INM-002          120           Casa  Medellin
2  INM-003           50    Apartamento      Cali
3  INM-004          250           Casa    Bogota
4  INM-005           45    Apartamento  Medellin


In [157]:
# Eliminación de espacios en la columna 'Ciudad' del DataFrame de ciudades
data_cities= data_cities.apply(
    lambda x: x.str.strip() if x.dtype == "object" else x
)

print("Datos de ciudades limpias:")
print(data_cities)

Datos de ciudades limpias:
  ciudad_ref  estrato_promedio riesgo_inversion
0     Bogota                 4             Bajo
1   Medellin                 5            Medio
2       Cali                 3             Alto
3  Cartagena                 6             Bajo


In [158]:
#Verificamos que los datos estén limpios y sean de su tipo para el siguiente paso que es la mezcla de estas tablas en uno solo DataFrame
print("Tipos de datos en data_price:")
print(data_price.dtypes)
print("Tipos de datos en data_properties:")
print(data_properties.dtypes)
print("Tipos de datos en data_cities:")
print(data_cities.dtypes)

Tipos de datos en data_price:
id_propiedad        str
Mes                 str
Precios         float64
dtype: object
Tipos de datos en data_properties:
ID                  str
area_metros       int64
Tipo_Propiedad      str
Ciudad              str
dtype: object
Tipos de datos en data_cities:
ciudad_ref            str
estrato_promedio    int64
riesgo_inversion      str
dtype: object


In [159]:
print("Datos de data_price:")
print(data_price)
print("Datos de data_properties:")
print(data_properties)
print("Datos de data_cities:")
print(data_cities)

Datos de data_price:
   id_propiedad       Mes  Precios
0       INM-001  Ene_2024    350.0
1       INM-002  Ene_2024    420.0
2       INM-003  Ene_2024      0.0
3       INM-004  Ene_2024    800.0
4       INM-005  Ene_2024    120.0
5       INM-001  Feb_2024    355.0
6       INM-002  Feb_2024    415.0
7       INM-003  Feb_2024    210.0
8       INM-004  Feb_2024    790.0
9       INM-005  Feb_2024    120.0
10      INM-001  Mar_2024    360.0
11      INM-002  Mar_2024    410.0
12      INM-003  Mar_2024    215.0
13      INM-004  Mar_2024    810.0
14      INM-005  Mar_2024    125.0
Datos de data_properties:
        ID  area_metros Tipo_Propiedad    Ciudad
0  INM-001           65    Apartamento    Bogota
1  INM-002          120           Casa  Medellin
2  INM-003           50    Apartamento      Cali
3  INM-004          250           Casa    Bogota
4  INM-005           45    Apartamento  Medellin
Datos de data_cities:
  ciudad_ref  estrato_promedio riesgo_inversion
0     Bogota                 

In [160]:
# Relizada la limpieza de los datos, se pone en marcha la mezcla de los dataframes en uno solo, se usara pd.concat()
# Primero se hace un merge entre data_price y data_properties usando 'id_propiedad' y 'ID' como claves de unión
data_merged = pd.merge(data_price, data_properties, left_on='id_propiedad', right_on='ID', how='inner')

# Se elimina la columna 'ID' redundante, ya que contiene la misma información que 'id_propiedad'
data_merged = data_merged.drop(columns=['ID'])

print(data_merged)


   id_propiedad       Mes  Precios  area_metros Tipo_Propiedad    Ciudad
0       INM-001  Ene_2024    350.0           65    Apartamento    Bogota
1       INM-002  Ene_2024    420.0          120           Casa  Medellin
2       INM-003  Ene_2024      0.0           50    Apartamento      Cali
3       INM-004  Ene_2024    800.0          250           Casa    Bogota
4       INM-005  Ene_2024    120.0           45    Apartamento  Medellin
5       INM-001  Feb_2024    355.0           65    Apartamento    Bogota
6       INM-002  Feb_2024    415.0          120           Casa  Medellin
7       INM-003  Feb_2024    210.0           50    Apartamento      Cali
8       INM-004  Feb_2024    790.0          250           Casa    Bogota
9       INM-005  Feb_2024    120.0           45    Apartamento  Medellin
10      INM-001  Mar_2024    360.0           65    Apartamento    Bogota
11      INM-002  Mar_2024    410.0          120           Casa  Medellin
12      INM-003  Mar_2024    215.0           50    

In [161]:
# Ahora se mezcla con 'data_cities' usando la columna 'Ciudad' como clave de unión
data_final = pd.merge(data_merged, data_cities, left_on='Ciudad',right_on='ciudad_ref', how='left')

#Se elimina la columna 'ciudad_ref' redundante, ya que contiene la misma información que 'Ciudad'
data_final = data_final.drop(columns=['ciudad_ref'])
print(data_final)


   id_propiedad       Mes  Precios  area_metros Tipo_Propiedad    Ciudad  \
0       INM-001  Ene_2024    350.0           65    Apartamento    Bogota   
1       INM-002  Ene_2024    420.0          120           Casa  Medellin   
2       INM-003  Ene_2024      0.0           50    Apartamento      Cali   
3       INM-004  Ene_2024    800.0          250           Casa    Bogota   
4       INM-005  Ene_2024    120.0           45    Apartamento  Medellin   
5       INM-001  Feb_2024    355.0           65    Apartamento    Bogota   
6       INM-002  Feb_2024    415.0          120           Casa  Medellin   
7       INM-003  Feb_2024    210.0           50    Apartamento      Cali   
8       INM-004  Feb_2024    790.0          250           Casa    Bogota   
9       INM-005  Feb_2024    120.0           45    Apartamento  Medellin   
10      INM-001  Mar_2024    360.0           65    Apartamento    Bogota   
11      INM-002  Mar_2024    410.0          120           Casa  Medellin   
12      INM-

In [162]:
# Se calcula el precio promedio por metro cuadrado para cada propiedad dividiendo el precio por el área
data_final = data_final.assign(
    Precio_m2=lambda x:x['Precios'] / x['area_metros'],
    Categoria = lambda x: np.where( (x['Precios'] > 400) | (x['area_metros'] > 100), 'Premium', 'Estándar' ) # Se asigna la categoría 'Premium' si el precio es mayor a 400 o el área es mayor a 100 metros cuadrados, de lo contrario se asigna 'Estándar'
)
# Se muestra el DataFrame final con la nueva columna de precio por metro cuadrado
print(data_final)

   id_propiedad       Mes  Precios  area_metros Tipo_Propiedad    Ciudad  \
0       INM-001  Ene_2024    350.0           65    Apartamento    Bogota   
1       INM-002  Ene_2024    420.0          120           Casa  Medellin   
2       INM-003  Ene_2024      0.0           50    Apartamento      Cali   
3       INM-004  Ene_2024    800.0          250           Casa    Bogota   
4       INM-005  Ene_2024    120.0           45    Apartamento  Medellin   
5       INM-001  Feb_2024    355.0           65    Apartamento    Bogota   
6       INM-002  Feb_2024    415.0          120           Casa  Medellin   
7       INM-003  Feb_2024    210.0           50    Apartamento      Cali   
8       INM-004  Feb_2024    790.0          250           Casa    Bogota   
9       INM-005  Feb_2024    120.0           45    Apartamento  Medellin   
10      INM-001  Mar_2024    360.0           65    Apartamento    Bogota   
11      INM-002  Mar_2024    410.0          120           Casa  Medellin   
12      INM-

In [163]:
# Se saca el resumen del primer trimestre (Q1)
resumen_q1 = data_final.pivot_table(
    index='Ciudad',
    columns='Tipo_Propiedad',
    values='Precio_m2',
    aggfunc='mean'
)
print("Resumen del primer trimestre (Q1):")
print(resumen_q1)

Resumen del primer trimestre (Q1):
Tipo_Propiedad  Apartamento      Casa
Ciudad                               
Bogota             5.461538  3.200000
Cali               2.833333       NaN
Medellin           2.703704  3.458333


In [164]:
# Conversion a CSV para mejor visualización de los datos finales
data_final.to_csv('archivos_finales/data_final.csv', index=False)